# Detect Language with Memory Sources

## Introduction

The goal is to recognize the language of a given text, 
from a known list of languages (Romanian and English).

We will build an information source model for each language, considering it as 
a source with memory of order $m=1$. This means we will 
estimate the probability of every letter given the previous letter.
This is very characteristic for each language, and it provides a simple way to
distinguish between them.


## Walkthrough

Given a large text file, we would like to build the transition matrix
which contains the probability of each letter given the previous letter.

We consider only the letters from the (English) alphabet, and we ignore the case of the letters.
This means there are 26 letters in all (from a to z). 

Since memory is of order 1, the state is **the previous letter**.
This means we have $N=26$ states, and we need to estimate a transition matrix of size $26 \times 26$.

We estimate the transition matrix by counting every pair of consecutive letters, 
then normalizing the counts to obtain probabilities.

### Count pairs of letters

In [22]:
import numpy as np

filename = "data/texten.txt"

# Read all text from then file
with open(filename, 'r') as file:
    text = file.read()

# Convert all text to lowercase:
text = text.lower()

# Prepare 26x26 matrix full of **ones**
# Why ones and not zeros?  Discuss later
T = np.ones((26, 26))

# Go through all the text
for i in range(len(text) - 1):

    # Previous character and current character
    prev = text[i]
    curr = text[i + 1]

    # We only consider letters
    if prev.isalpha() and curr.isalpha():

        # Convert characters to numbers = offset from 'a'
        row = ord(prev) - ord('a')
        col = ord(curr) - ord('a')

        # Increment counter at position (row, col)
        T[row, col] += 1

Let's look at the matrix we have now. The display will show the values row by row.

In [5]:
T

array([[  0.,  43.,  22.,  25.,   2.,   5.,  46.,   0.,  30.,   0.,  13.,
         53.,  20., 190.,   0.,  13.,   0.,  78.,  63., 104.,  14.,  41.,
          8.,   0.,  24.,   0.],
       [ 39.,   2.,   0.,   1.,  46.,   0.,   0.,   1.,   8.,   0.,   0.,
         14.,   0.,   0.,  16.,   0.,   0.,  38.,   2.,   0.,  32.,   0.,
          0.,   0.,  15.,   0.],
       [ 37.,   0.,   2.,   0.,  52.,   0.,   0.,  46.,  20.,   0.,   9.,
          7.,   0.,   0.,  45.,   0.,   0.,   3.,   0.,  19.,   8.,   0.,
          0.,   0.,   6.,   0.],
       [ 18.,   0.,   0.,   1.,  71.,   0.,   4.,   1.,  23.,   0.,   0.,
          3.,   0.,   2.,  36.,   0.,   0.,   8.,  16.,   0.,  16.,   1.,
          1.,   0.,   6.,   0.],
       [ 52.,   2.,  18.,  46.,  35.,   8.,   4.,   2.,  13.,   1.,   1.,
         60.,  25., 108.,   3.,   4.,   3., 215.,  86.,  36.,   6.,  28.,
          7.,   9.,   9.,   0.],
       [ 17.,   0.,   0.,   0.,  19.,  14.,   0.,   0.,  21.,   0.,   0.,
          4.,   0.,  

Let's see the counts when the first letter is `a` (i.e. row 0 in the matrix):


In [6]:
T[0,:]

array([  0.,  43.,  22.,  25.,   2.,   5.,  46.,   0.,  30.,   0.,  13.,
        53.,  20., 190.,   0.,  13.,   0.,  78.,  63., 104.,  14.,  41.,
         8.,   0.,  24.,   0.])

Let's see the counts where the second letter is `h` (i.e. column number (`h` - `a`) in the matrix):

In [7]:
T[:,ord('h') - ord('a')]

array([  0.,   1.,  46.,   1.,   2.,   0.,  37.,   0.,   0.,   0.,   0.,
         0.,   0.,   3.,   1.,   1.,   0.,   1.,  31., 313.,   0.,   0.,
        54.,   0.,   0.,   0.])

Let's locate the pair of letters with the highest count.

In [11]:
maxpos = np.argmax(T)
maxrow, maxcol = np.unravel_index(maxpos, T.shape)
maxrow, maxcol

(19, 7)

This means the 20th row and 8th column in the matrix (Python starts numbering from 0), 
which correspond to the pair `t` (20th letter) and `h` (8th letter).

The most frequent pair of letters is `th`.

### Normalize the counts

We need to turn counts into probabilities. 

To do this, we divide each row in the matrix **by the sum of that row**, so that the row sums to 1.

In [14]:
T = np.array([row  / sum(row) for row in T])

Let's look at the first row and check that it's sum is 1:

In [17]:
print(T[0,:])
print(f"Sum of row 0 is {sum(T[0,:])}")

[0.         0.05415617 0.02770781 0.03148615 0.00251889 0.00629723
 0.05793451 0.         0.03778338 0.         0.0163728  0.06675063
 0.02518892 0.23929471 0.         0.0163728  0.         0.09823678
 0.07934509 0.13098237 0.01763224 0.05163728 0.01007557 0.
 0.0302267  0.        ]
Sum of row 0 is 1.0


### Compute the probability of a sequence given the model

In order to compute the probability of a sequence of letters, 
we multiply the transition probabilities for each pair of consecutive letters.
We shall ignore the probability of the first letter (which doesn't have a previous letter), for simplicity.

For example, for the sequence `hello", we need to multiply the following probabilities:

$$
\begin{align*}
P &= P_{he} \cdot P_{el} \cdot P_{ll} \cdot P_{lo} \\
&= P(h|e) \cdot P(e|l) \cdot P(l|l) \cdot P(l|o) \\
\end{align*}
$$

Because the probabilities are small, the result will quickly become too small
to be represented by a floating point number.
To avoid this, we use log-probabilities, and we add them instead of multiplying them:
$$
\begin{align*}
\log(P_{he}) + \log(P_{el}) + \log(P_{ll}) + \log(P_{lo}) \\
\end{align*}
$$
This works because all we want is to compare the result with one matrix to the result with another matrix,
and the logarithm is a monotonically increasing function, so we can compare the log-probabilities directly
instead of the probabilities themselves.

In [18]:
text = "hello"
logprob = 0
for i in range(len(text) - 1):
    prev = text[i]
    curr = text[i + 1]
    if prev.isalpha() and curr.isalpha():
        row = ord(prev) - ord('a')
        col = ord(curr) - ord('a')
        logprob = logprob + np.log(T[row, col])
logprob

-6.377369106514646

## Experiments


We will encapsulate the code in some functions, so that we can reuse it conveniently

In [25]:
def compute_transition_matrix(filename):
    with open(filename, 'r') as file:
        text = file.read()
    text = text.lower()
    T = np.ones((26, 26))                   ### 1's not 0's
    for i in range(len(text) - 1):
        prev = text[i]
        curr = text[i + 1]
        if prev.isalpha() and curr.isalpha():
            row = ord(prev) - ord('a')
            col = ord(curr) - ord('a')
            T[row, col] += 1
        T = np.array([row  / sum(row) for row in T])
    return T

def compute_logprob(text, T):
    text = text.lower()
    logprob = 0
    for i in range(len(text) - 1):
        prev = text[i]
        curr = text[i + 1]
        if prev.isalpha() and curr.isalpha():
            row = ord(prev) - ord('a')
            col = ord(curr) - ord('a')
            logprob += np.log(T[row, col])
    return logprob

def get_language(text, T_en, T_ro):
    logprob_en = compute_logprob(text, T_en)
    logprob_ro = compute_logprob(text, T_ro)
    return "English" if logprob_en > logprob_ro else "Romanian"



Let's compute the transition matrix for English and Romanian.

In [34]:
Ten = compute_transition_matrix("data/texten.txt")
Tro = compute_transition_matrix("data/textro.txt")

Now let's guess the language:

In [35]:
sentence = "Salut! Ce mai faci?"
get_language(sentence, Ten, Tro)

'Romanian'

In [36]:
sentence = "Scooby-Doo, where are you?"
get_language(sentence, Ten, Tro)

'English'